In [0]:
import re

In [0]:

dbutils.widgets.text("file_name", "")
file_name = dbutils.widgets.get("file_name")

match = re.match(r"([^-]+)-bordereaux.*", file_name)
if match:
    table_name = match.group(1)
else:
    raise ValueError("Invalid file name format. Expected format: <table>-bordereaux-<year>.csv")

excel_file_path = f"/mnt/bordereaux/bronze/{file_name}"
delta_table = f"hive_metastore.bordereaux.{table_name}"

df = spark.read.csv(
    excel_file_path,
    header=True,
    inferSchema=True,
    multiLine=True,         # In case some fields have line breaks
    quote='"',              # Properly handle comma-separated quoted fields
    escape='"',             # Handle quotes inside quoted fields
    mode="PERMISSIVE"       # Handle malformed lines gracefully
)


dbutils.widgets.text("file_name", "")
file_name = dbutils.widgets.get("file_name")

match = re.match(r"([^-]+)-bordereaux.*", file_name)
if match:
    table_name = match.group(1)
else:
    raise ValueError("Invalid file name format. Expected format: <table>-bordereaux-<year>.csv")

excel_file_path = f"/mnt/bordereaux/bronze/{file_name}"
delta_table = f"hive_metastore.bordereaux.{table_name}"

df = spark.read.csv(
    excel_file_path,
    header=True,
    inferSchema=True,
    multiLine=True,         # In case some fields have line breaks
    quote='"',              # Properly handle comma-separated quoted fields
    escape='"',             # Handle quotes inside quoted fields
    mode="PERMISSIVE"       # Handle malformed lines gracefully
)

# Drop the table if it exists
spark.sql(f"DROP TABLE IF EXISTS {delta_table}")

# Then create/load the new table
df.write.format("delta") \
    .mode("overwrite") \
    .option("truncate", "true") \
    .saveAsTable(delta_table)

